In [1]:
import torch
from torch import nn
import torch.fx as fx
import torch.nn.functional as F932
import torchvision
import trtorch
#from det2_test_loader import get_test_loader 
import os

In [2]:
model = torch.jit.load("/falldetector/detectron2/output_tracing_float/model.ts")

In [3]:
compile_settings = {
    "input_shapes": [
        (3,800,1202)
#         {
#             "min" : [3, 480,640],
#             "opt" : [3, 1080,1920],
#             "max" : [3, 1440, 2560],
#         }
    ],
    "op_precision": torch.half, # Run with FP16
    "truncate_long_and_double": True,
    "device": {
        "device_type": trtorch.DeviceType.DLA, # Type of device to run engine on (for DLA use trtorch.DeviceType.DLA)
        "gpu_id": 0, # Target gpu id to run engine (Use Xavier as gpu id for DLA)
        "dla_core": 0, # (DLA only) Target dla core id to run engine
        "allow_gpu_fallback": True, # (DLA only) Allow layers unsupported on DLA to run on GPU
    }
}

trt_ts_module = trtorch.compile(model.float(), compile_settings)
print('COMPILED!!')
input_data = torch.randn((3, 1080, 1920))
input_data = input_data.half().to("cuda")

input_data = input_data.half()
result = trt_ts_module(input_data)
torch.jit.save(trt_ts_module, "trt_ts_module.ts")

RuntimeError: [enforce fail at core/conversion/converters/impl/index.cpp:53] Expected const_layer to be true but got false
Unable to create constant layer from node: %pooler_fmt_boxes_level0.2 : Tensor = aten::index(%1220, %1202) # /falldetector/detectron2/detectron2/modeling/poolers.py:246:0



## os.chdir('/falldetector/detectron2')
testing_dataloader = get_test_loader()
os.chdir('/falldetector/trt-converter')

calibrator = trtorch.ptq.DataLoaderCalibrator(testing_dataloader,
                                              cache_file='./calibration.cache',
                                              use_cache=False,
                                              algo_type=trtorch.ptq.CalibrationAlgo.ENTROPY_CALIBRATION_2,
                                              device=torch.device('cuda:0'))

compile_spec = {
         "input_shapes": [
            {
            "min" : [1, 3,480,640],
            "opt" : [1, 3,1080,1920],
            "max" : [1, 3, 1440, 2560],
            }
        ],
         "op_precision": torch.int8,
         "calibrator": calibrator,
        "truncate_long_and_double": True,
         "device": {
             "device_type": trtorch.DeviceType.GPU,
             "gpu_id": 0,
             "dla_core": 0,
             "allow_gpu_fallback": False,
             "disable_tf32": False
         }
     }
trt_mod = trtorch.compile(model, compile_spec)

In [ ]:
type(model)

In [1]:
import tensorrt
tensorrt.__version__

'7.1.3.0'